# SORA with Azure AI Foundry
> https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/video-generation

In [1]:
import base64
import datetime
import os
import requests
import sys
import time

from dotenv import load_dotenv
from IPython.display import Video

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 30-May-2025 09:19:09


In [4]:
SORA_DIR = "videos"

os.makedirs(SORA_DIR, exist_ok=True)

In [5]:
load_dotenv("azure.env")

endpoint = os.environ['AZURE_OPENAI_ENDPOINT']
api_key = os.environ['AZURE_OPENAI_API_KEY']

model = "sora"

# Function

In [6]:
def sora(prompt, width=480, height=480, n_seconds=5, n_variants=1):
    """
    Generates a video based on the given prompt using the SORA model.

    Parameters:
    prompt (str): The text prompt to generate the video.
    width (int): The width of the video. Supported values are 480, 854, 720, 1080, and 1920.
    height (int): The height of the video. Supported values are 480, 854, 720, 1080, and 1920.
    n_seconds (int): The duration of the video in seconds. Must be between 1 and 20 seconds.
    n_variants (int): The number of video variants to generate.
    
    Returns:
    str: The filename of the generated video.

    Raises:
    Exception: If the video generation job fails or no generations are found.
    """
    start = time.time()

    api_version = 'preview'
    headers = {"api-key": api_key, "Content-Type": "application/json"}

    idx = datetime.datetime.today().strftime('%d%b%Y_%H%M%S')
    output_filename = os.path.join(SORA_DIR, f"sora_{idx}.mp4")

    # 1. Create a video generation job
    create_url = f"{endpoint}/openai/v1/video/generations/jobs?api-version={api_version}"
    body = {
        "prompt": prompt,
        "width": width,  # 480x480, 480x854, 854x480, 720x720, 720x1280, 1280x720, 1080x1080, 1080x1920, 1920x1080.
        "height": height,  # 480x480, 480x854, 854x480, 720x720, 720x1280, 1280x720, 1080x1080, 1080x1920, 1920x1080.
        "n_seconds": n_seconds,  # between 1 and 20 seconds
        "n_variants": n_variants,
        "model": model,  # SORA model
    }
    response = requests.post(create_url, headers=headers, json=body)
    response.raise_for_status()

    now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
    print(f"{now} Full response JSON:", response.json())
    print()
    
    job_id = response.json()["id"]
    now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
    print(f"{now} Job created: {job_id}")

    # 2. Poll for job status
    status_url = f"{endpoint}/openai/v1/video/generations/jobs/{job_id}?api-version={api_version}"
    status = None

    while status not in ("succeeded", "failed", "cancelled"):
        time.sleep(5)  # Wait before polling again
        status_response = requests.get(status_url, headers=headers).json()
        status = status_response.get("status")
        now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
        print(f"{now} Job status: {status}")

    # 3. Retrieve generated video
    if status == "succeeded":
        generations = status_response.get("generations", [])
        
        if generations:
            now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
            print(f"\n{now} ✅ Done. Video generation succeeded.")
            generation_id = generations[0].get("id")
            video_url = f"{endpoint}/openai/v1/video/generations/{generation_id}/content/video?api-version={api_version}"
            video_response = requests.get(video_url, headers=headers)
            
            if video_response.ok:
                # Downloading the video
                with open(output_filename, "wb") as file:
                    file.write(video_response.content)
                    print(f"\nSORA Generated video saved: '{output_filename}'")

                elapsed = time.time() - start
                minutes, seconds = divmod(elapsed, 60)
                print(f"Done in {minutes:.0f} minutes and {seconds:.0f} seconds")
                
                return output_filename
        else:
            raise Exception("Error. No generations found in job result.")
    else:
        raise Exception(f"Error. Job did not succeed. Status: {status}")

## Examples

In [7]:
prompt = "Young boy and his father playing together in the ocean on the beach"

generated_video = sora(prompt, width=480, height=480, n_seconds=5, n_variants=1)

30-May-2025 09:19:09 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwg7sxhcf2wsj89eb97sxen5', 'status': 'queued', 'created_at': 1748596749, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'Young boy and his father playing together in the ocean on the beach', 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 480, 'width': 480, 'failure_reason': None}

30-May-2025 09:19:09 Job created: task_01jwg7sxhcf2wsj89eb97sxen5
30-May-2025 09:19:15 Job status: running
30-May-2025 09:19:20 Job status: running
30-May-2025 09:19:26 Job status: processing
30-May-2025 09:19:31 Job status: succeeded

30-May-2025 09:19:31 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos/sora_30May2025_091909.mp4'
Done in 0 minutes and 26 seconds


In [8]:
Video(generated_video)

In [9]:
prompt = "An image of a realistic cloud that spells 'SORA with birds around'"

generated_video = sora(prompt, width=480, height=480, n_seconds=5, n_variants=1)

30-May-2025 09:19:35 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwg7tpgzf7esfthsn3011ej1', 'status': 'queued', 'created_at': 1748596775, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': "An image of a realistic cloud that spells 'SORA with birds around'", 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 480, 'width': 480, 'failure_reason': None}

30-May-2025 09:19:35 Job created: task_01jwg7tpgzf7esfthsn3011ej1
30-May-2025 09:19:40 Job status: queued
30-May-2025 09:19:46 Job status: running
30-May-2025 09:19:51 Job status: succeeded

30-May-2025 09:19:51 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos/sora_30May2025_091935.mp4'
Done in 0 minutes and 19 seconds


In [10]:
Video(generated_video)

In [11]:
prompt = "Several giant wooly mammoths approach treading through a snowy meadow, their long wooly fur lightly blows in the wind as they walk, snow covered trees and dramatic snow capped mountains in the distance, mid afternoon light with wispy clouds and a sun high in the distance creates a warm glow, the low camera view is stunning capturing the large furry mammal with beautiful photography, depth of field."

generated_video = sora(prompt, width=480, height=480, n_seconds=5, n_variants=1)

30-May-2025 09:19:54 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwg7v8saesabp2bt89abhts0', 'status': 'queued', 'created_at': 1748596794, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'Several giant wooly mammoths approach treading through a snowy meadow, their long wooly fur lightly blows in the wind as they walk, snow covered trees and dramatic snow capped mountains in the distance, mid afternoon light with wispy clouds and a sun high in the distance creates a warm glow, the low camera view is stunning capturing the large furry mammal with beautiful photography, depth of field.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 480, 'width': 480, 'failure_reason': None}

30-May-2025 09:19:54 Job created: task_01jwg7v8saesabp2bt89abhts0
30-May-2025 09:19:59 Job status: preprocessing
30-May-2025 09:20:05 Job status: running
30-May-2025 09:20:10 Job status: succeeded

30-May-2025 09:20:10 ✅ Done. Video generation succeeded.

SO

In [12]:
Video(generated_video)

In [13]:
prompt = "Paris submerged like Atlantis. Fish, whales, sea turtles and sharks swim through the streets of Paris."

generated_video = sora(prompt, width=480, height=480, n_seconds=5, n_variants=1)

30-May-2025 09:20:13 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwg7vv6nfnq885kab1z65yww', 'status': 'queued', 'created_at': 1748596813, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'Paris submerged like Atlantis. Fish, whales, sea turtles and sharks swim through the streets of Paris.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 480, 'width': 480, 'failure_reason': None}

30-May-2025 09:20:13 Job created: task_01jwg7vv6nfnq885kab1z65yww
30-May-2025 09:20:18 Job status: preprocessing
30-May-2025 09:20:23 Job status: running
30-May-2025 09:20:29 Job status: succeeded

30-May-2025 09:20:29 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos/sora_30May2025_092012.mp4'
Done in 0 minutes and 20 seconds


In [14]:
Video(generated_video)

In [15]:
prompt = "A instructional cooking session for homemade gnocchi hosted by a grandmother social media influencer set in a rustic Tuscan country kitchen with cinematic lighting."

generated_video = sora(prompt, width=480, height=480, n_seconds=5, n_variants=1)

30-May-2025 09:20:32 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwg7webwe8sb1rhd94wq5gj4', 'status': 'queued', 'created_at': 1748596832, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'A instructional cooking session for homemade gnocchi hosted by a grandmother social media influencer set in a rustic Tuscan country kitchen with cinematic lighting.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 480, 'width': 480, 'failure_reason': None}

30-May-2025 09:20:32 Job created: task_01jwg7webwe8sb1rhd94wq5gj4
30-May-2025 09:20:38 Job status: preprocessing
30-May-2025 09:20:43 Job status: running
30-May-2025 09:20:48 Job status: processing
30-May-2025 09:20:54 Job status: succeeded

30-May-2025 09:20:54 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos/sora_30May2025_092032.mp4'
Done in 0 minutes and 24 seconds


In [16]:
Video(generated_video)

## Videos

In [17]:
!ls $SORA_DIR -lh

total 15M
-rwxrwxrwx 1 root root 4.7M May 30 09:19 sora_30May2025_091909.mp4
-rwxrwxrwx 1 root root 2.1M May 30 09:19 sora_30May2025_091935.mp4
-rwxrwxrwx 1 root root 2.1M May 30 09:20 sora_30May2025_091953.mp4
-rwxrwxrwx 1 root root 3.0M May 30 09:20 sora_30May2025_092012.mp4
-rwxrwxrwx 1 root root 3.0M May 30 09:20 sora_30May2025_092032.mp4
